In [1]:
from time import sleep
from IPython.display import clear_output


MIN_VAL = 3
MAX_VAL = 8
MS = 0.5


header4 = """
 _____                               __   _   _                   _ 
|_   _|                             / _| | | | |                 (_)
  | | _____      _____ _ __    ___ | |_  | |_| | __ _ _ __   ___  _ 
  | |/ _ \ \ /\ / / _ \ '__|  / _ \|  _| |  _  |/ _` | '_ \ / _ \| |
  | | (_) \ V  V /  __/ |    | (_) | |   | | | | (_| | | | | (_) | |
  \_/\___/ \_/\_/ \___|_|     \___/|_|   \_| |_/\__,_|_| |_|\___/|_|

"""

disks_input = int(input('[+] Saisir le nombre de disques : '))


def ascii_drawing(disk_size:int, choice:int) -> str:
    """Draw a disk with ascii chars - ####"""

    disks = disks_input

    # Width du disque impaire
    max_width = disks * 2 - 1 
    disk = disk_size * 2 - 1 

    # Nombre d'espace à rajouter pour centrer le disque
    nb_blanks = max_width - disk
    blanks = ' ' *  int((nb_blanks / 2))

    # Concaténation du disque et des espaces
    if choice == 0:
        if disk_size % 2 == 0:
            draw = f"{blanks}{'#' * disk}{blanks}"

        elif disk_size % 2 != 0:
            draw = f"{blanks}{'#' * disk}{blanks}"

    # Concaténation de la tour et des espaces
    elif choice == 1:
        draw = f"{blanks}{'|' * disk}{blanks}"

    return draw


def disk_index(tower:int, i:int, towers:int) -> str:
    """Test index out of orange - return ' ' or index"""

    try:
        width = towers[tower][i]
        disk = ascii_drawing(width, 0)
        return disk

    except IndexError:
        width = 1
        stick = ascii_drawing(width, 1)
        return stick


def render_game(towers:list, move:int) -> print:
    """Print the game in cli"""

    disks = disks_input
    output = header4 + "\n"
    sleep(MS)

    # Draw the puzzle disks
    for i in reversed(range(disks)):
        line = "\t%s\t%s\t%s" % (
            disk_index(0, i, towers),
            disk_index(1, i, towers),
            disk_index(2, i, towers),
        )
        output += f"\n{line}"

    blanks = disks * 2 - 1

    # Draw towers bottom line
    stick = ' ' * int((blanks - len("|")) / 2)
    platform = ' ' * int((blanks - len("___")) / 2)
    base2 = f"\t{platform}‾‾‾{platform}\t{platform}‾‾‾{platform}\t{platform}‾‾‾{platform}\n"
    moves = f"\n[+] Move played : {move}/{min_movements()}" 
    output += f"\n{base2}\n{moves}"

    return output


def range_int_type(arg:str) -> int:
    """Type function for argparse - int within some predefindes bounds"""

    try:
        value = int(arg)
    except ValueError:
        raise argparse.ArgumentTypeError("Must be a integer digit.")

    if value < MIN_VAL or value > MAX_VAL:
        raise argparse.ArgumentTypeError(f"Argument must be < {MAX_VAL} and > {MIN_VAL}")

    return value


def min_movements() -> int:
    """Return the minimal movement to end the game"""

    n = disks_input
    minmov = 2 ** n - 1

    return minmov



# Top: Rendering
# ====================================================================
# Bottom: Solving puzzle



def select_disk(towers:list, disk_moved:int, excludes:list) -> None:
    """Pick a disk"""

    for count, tower in enumerate(towers):

        # Check if tower is not empty
        if not tower:
            continue

        else:
            disk = tower[-1]        

            # Check if disk has been tested
            if disk in excludes:
                continue

            else:
                # Check if picked disk has moved
                if disk != disk_moved:
                    return (count, disk)


def moving_disk(towers:list, tower:list, disk:tuple, move:int) -> None:
    """Move a disk from a tower to another"""

    tower.append(disk[1])
    towers[disk[0]].pop()
    clear_output(wait=True)
    print(render_game(towers, move))
    sleep(MS)

    return


def init_game() -> list:
    """Init the puzzle board"""

    nb_disks = disks_input
    game = [[disk + 1 for disk in reversed(range(nb_disks))], [], []]

    return game


def first_move(towers:list, x:int) -> None:
    """Move the first disk"""

    towers[-x].append(towers[0].pop())
    disk_moved = towers[-x][0]

    return disk_moved


def main() -> None:
    """Main job"""

    # Initializing puzzle
    towers = init_game()
    excludes = []
    goal = towers[0] 

    # Move 0 - Is not even
    if len(goal) % 2 != 0:
        disk_moved = first_move(towers, 1)
        x = 0

    # Move 0 - Is even
    elif len(goal) % 2 == 0:
        disk_moved = first_move(towers, 2)
        x = 1

    # First move
    move = 1
    clear_output(wait=True)
    print(render_game(towers, move))
    sleep(MS)

    # Resolving the puzzle..
    while towers[-1] != goal:

        disk = select_disk(towers, disk_moved, excludes)

        if disk is None:
            break

        # Placing disk..
        for count, tower in enumerate(towers):

            # Pass if tower is the same where the disk was picked
            if count == disk[0]:
                continue

            # If tower is empty - Check if the disk can be placed here
            if not tower:
                if towers.index(tower) + x % 2 != disk[1] % 2:
                    disk_moved = disk[1]
                    excludes.clear()
                    move += 1
                    moving_disk(towers, tower, disk, move)
                    
                    break
            
            # If tower is not empty - Check if the tower and disk are not both even
            # & Check if the top disk is smaller than the picked one
            else:
                if disk[1] < tower[-1] and disk[1] % 2 != tower[-1] % 2:
                    disk_moved = disk[1]
                    excludes.clear()
                    move += 1
                    moving_disk(towers, tower, disk, move)

                    break

                else:
                    continue

        # If disk cant be placed - Add into excludes disks
        excludes.append(disk[1])
        
    
main()




 _____                               __   _   _                   _ 
|_   _|                             / _| | | | |                 (_)
  | | _____      _____ _ __    ___ | |_  | |_| | __ _ _ __   ___  _ 
  | |/ _ \ \ /\ / / _ \ '__|  / _ \|  _| |  _  |/ _` | '_ \ / _ \| |
  | | (_) \ V  V /  __/ |    | (_) | |   | | | | (_| | | | | (_) | |
  \_/\___/ \_/\_/ \___|_|     \___/|_|   \_| |_/\__,_|_| |_|\___/|_|



	   |   	   |   	   #   
	   |   	   |   	  ###  
	   |   	   |   	 ##### 
	   |   	   |   	#######
	  ‾‾‾  	  ‾‾‾  	  ‾‾‾  


[+] Move played : 15/15
